# Introduction

This notebook is intended to learn linear programming by solving several problems using Python solver. Herein, the [PuLP](https://coin-or.github.io/pulp/) library is chosen as a solver packages. 

Linear programming often involves maximising or minimising an objective function which subject to several constraints. To solve optimization problems, the following steps are generally carried out:
1. Clearly make the problem description
2. Formulate the problem into mathematical formulation
3. Solve the mathematical program
4. Perform post-optimal analysis
5. Present the solution and analysis

The problem used in this notebook will be varied ranging from text book, online source, and real problems. 

# Problem definition
The problems from online source were taken from:
* https://github.com/tstran155/Linear-Programming-Optimization-With-Python.syntax
* https://github.com/Gurobi

The problems from text book were taken from:
* Applied Mathematical Programming
* Supply Chain Management: Strategy, Planning, and Operation

# Example 3
See the other series of this notebook:
* [Example 1](https://jupyter.uit.no/hub/user-redirect/lab/tree/learnProgramming/linearProgramming/LP-example1.ipynb)
* [Example 2](https://jupyter.uit.no/hub/user-redirect/lab/tree/learnProgramming/linearProgramming/LP-example2.ipynb)

## Problem 1
**Agregate Planning Red Tomato Tools** - The problem is taken from *Supply Chain Management: Strategy, Planning, and Operation - Chapter 8* 

The demand for Red Tomato’s gardening tools from consumers is highly seasonal, peaking in the
spring as people plant their gardens. This seasonal demand ripples up the supply chain from the
retailer to Red Tomato, the manufacturer. The options Red Tomato has for handling the seasonality are adding workers during the peak season, subcontracting out some of the work, building up inventory during the slow months, or building up a backlog of orders that will be delivered late to customers. To determine how to best use these options through an aggregate plan, Red Tom to’s vice president of supply chain starts with the first task—building a demand forecast. Although Red Tomato could attempt to forecast this demand itself, a much more accurate forecast comes from a collaborative process used by both Red Tomato and its retailers to produce the forecast shown in Table 8-2. It is important that this demand account for the product mix that is expected to sell and be in terms of aggregate units defined earliear.

| Month | Demand Forecast |
| -- | -- |
| January | 1600 |
| February | 3000 |
| March | 3200 |
| April | 3800 |
| May | 2200 |
| June | 2200 | 

---

| Item | Cost |
| -- | -- |
| Material Cost | $\$$10/unit |
| Inventory Holding Cost | $\$$2/unit/month |
| Marginal Cost of Stockout | $\$$5/unit/month |
| Hiring and Training Cost | $\$$300/worker |
| Layoff Cost | $\$$500/worker |
| Labour hours required | 4/unit |
| Regular time cost | $\$$4/hour |
| Overtime cost | $\$$6/hour |
| Cost of subscontracting | $\$$30/unit | 


Red Tomato sells each tool through retailers for $\$$40. The company has a starting inventory in January of 1,000 tools. At the beginning of January, the company has a workforce of 80 employees. The plant has a total of 20 working days in each month, and each employee earns $\$$4 per hour regular time. Each employee works eight hours per day on straight time and the rest on overtime. As discussed previously, the capacity of the production operation is determined primarily by the total labor hours worked. Therefore, machine capacity does not limit the capacity of the production operation. Because of labor rules, no employee works more than 10 hours of overtime per month. The various costs are shown in Table 8-3. It is important that the costs and labor hours are in aggregate units, as discussed in Section 8.2.

Currently, Red Tomato has no limits on subcontracting, inventories, and tockouts/backlog. All stockouts are backlogged and supplied from the following months’ production. Inventory costs are incurred on the ending inventory in the month. The supply cha n manager’s goal is to obtain the optimal aggregate plan that allows Red Tomato to end June  ith at least 500 units (i.e., no stockouts at the end of June and at least 500 units in inventory).


### Mathematical formulation

#### Decicion Variables
$ W_t = $ workforce size for Month $t, t=1, \dots , 6$

$ H_t = $ number of employees hired at the beginning of Month $t, t=1, \dots , 6$

$ L_t = $ number of employees laid off at the beginning of Month $t, t=1, \dots , 6$

$ P_t = $ number of units produced in Month $t, t=1, \dots , 6$

$ I_t = $ inventory at the end of Month $t, t=1, \dots , 6$

$ S_t = $ number of units stocked out at the end of Month $t, t=1, \dots , 6$

$ C_t = $ number of units subcontracted for Month $t, t=1, \dots , 6$

$ O_t = $ number of overtime hours worked in Month $t, t=1, \dots , 6$

#### Objective Function
Denote the demand in Period t by Dt. The values of Dt are as specified by the demand forecast in Table 8-2. The objective function is to minimize the total cost (equivalent to maximizing total profit as all demand is to be satisfied) incurred during the planning horizon. The cost incurred has the following components:* Regular-time labor cost
  Workers are paid a regular time wage of $\$$640 ($\$$4/hour X 8 hours/day X 20 days/month) per month. Because $W_t$ is the number of workers in period $t$, the regular-time labor cost over the planning horizon is given by:
  $$\text{Regular-time labor cost} = \sum_{t=1}^6 640W_t$$  
* Overtime labor cost
  $$\text{Overtime labor cost} = \sum_{t=1}^6 6O_t$$  
* Cost of hiring and layoffs
  $$\text{Cost of hiring and layoffs} = \sum_{t=1}^6 300H_t + \sum_{t=1}^6 500L_t$$  
* Cost of holding inventory and stockout
  $$\text{Cost of holding inventory and stockout} = \sum_{t=1}^6 2I_t + \sum_{t=1}^6 5S_t$$
* Cost of material and subcontracting
  $$\text{Cost of of material and subcontracting} = \sum_{t=1}^6 10P_t + \sum_{t=1}^6 30C_t$$

The total cost incurred during the planning horizon is the sum of aforementioned costs and can be expressed as:
\begin{equation}
\text{Minimize} \quad Z = \sum_{t=1}^6 640W_t + \sum_{t=1}^6 6O_t + \sum_{t=1}^6 300H_t + \sum_{t=1}^6 500L_t + \sum_{t=1}^6 2I_t + \sum_{t=1}^6 5S_t + \sum_{t=1}^6 10P_t + \sum_{t=1}^6 30C_t
\end{equation}

#### Constraints
1. Workforce, hiring, and layoffs constraints
   $$W_t = W_{t-1} + H_t - L_t \quad \text{for} \quad t=1, \dots, 6$$
2. Capacity constraints
   $$P_t \leq 40W_t + \dfrac{O_t}{4} \quad \text{for} \quad t=1, \dots, 6$$
3. Inventory balance constraints
   $$I_{t-1} + P_t + C_t = D_t + S_{t-1} + I_t - S_t \quad \text{for} \quad t=1, \dots, 6$$
   $$I_0 = 1000, I_6 \geq 500, S_0 = 0, S_6 = 0$$
5. Overtime limit constraints
   $$O_t = 10W_t \quad \text{for} \quad t=1, \dots, 6$$


#### Solution

In [1]:
pip install pulp

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pulp
pulp.listSolvers()

['GLPK_CMD',
 'PYGLPK',
 'CPLEX_CMD',
 'CPLEX_PY',
 'GUROBI',
 'GUROBI_CMD',
 'MOSEK',
 'XPRESS',
 'XPRESS',
 'XPRESS_PY',
 'PULP_CBC_CMD',
 'COIN_CMD',
 'COINMP_DLL',
 'CHOCO_CMD',
 'MIPCL_CMD',
 'SCIP_CMD',
 'HiGHS_CMD']

In [3]:
solver_list = pulp.listSolvers(onlyAvailable=True)
solver_list

['GLPK_CMD', 'PULP_CBC_CMD']

In [4]:
# add other solver
pulp.getSolver("CPLEX_CMD")
solver_list = pulp.listSolvers(onlyAvailable=True)
solver_list

['GLPK_CMD', 'PULP_CBC_CMD']

In [5]:
d = [0, 1600, 3000, 3200, 3800, 2200, 2200]
for i in range(1,7):
    print(i, d[i])

1 1600
2 3000
3 3200
4 3800
5 2200
6 2200


In [41]:
# 1st alternative
# create a model
from pulp import *
model = LpProblem("Red Tomato Problem", LpMinimize)

# define variables 
w = [LpVariable(f"workforce_{t}", lowBound =0, cat="Integer") for t in range (1,7)]
h = [LpVariable(f"no_hired_{t}", lowBound =0, cat="Integer") for t in range (1,7)]
l = [LpVariable(f"no_laidoff_{t}", lowBound =0, cat="Integer") for t in range (1,7)]
p = [LpVariable(f"product_{t}", lowBound =0, cat="Integer") for t in range (1,7)]
i = [LpVariable(f"inventory_{t}", lowBound =0, cat="Integer") for t in range (1,7)]
s = [LpVariable(f"stockout_{t}", lowBound =0, cat="Integer") for t in range (1,7)]
c = [LpVariable(f"subcontract_{t}", lowBound =0, cat="Integer") for t in range (1,7)]
o = [LpVariable(f"overtime_{t}", lowBound =0, cat="Integer") for t in range (1,7)]

# demand data
d = [1600, 3000, 3200, 3800, 2200, 2200]

# Objective function
model += (
    lpSum([640 * w[t] for t in range(6)]) + #labour cost
    lpSum([6 * o[t] for t in range(6)]) + #overtime cost
    lpSum([300 * h[t] for t in range(6)]) + #hiring cost
    lpSum([500 * l[t] for t in range(6)]) + #layoffs cost
    lpSum([2 * i[t] for t in range(6)]) + #inventory cost
    lpSum([5 * s[t] for t in range(6)]) + #stockout cost
    lpSum([10 * p[t] for t in range(6)]) + #materials cost
    lpSum([30 * c[t] for t in range(6)]), #subcontract cost
    "Total cost"
)

# constraints workforce
for t in range(6):
    if t == 0:
        model += 80 + h[t] - l[t] == w[t]
    else:
        model += w[t-1] + h[t] - l[t] == w[t]

# constraints capacity
for t in range(6):
    model += p[t] <= 40 * w[t] - o[t] * 0.25

# constraints balance inventory
for t in range(6):
    if t == 0:
        model += 1000 + p[t] + c[t] == d[t] + 0 + i[t] - s[t]
    else:
        model += i[t-1] + p[t] + c[t] == d[t] + s[t-1] + i[t] - s[t]

model += i[5] >= 500
model += s[5] == 0

# constraints overtime
for t in range(6):
    model += o[t] <= 10 * w[t]

print(model)

# Run the solver
model.writeLP("models/LP3-prob1-model1.lp")
model.solve()

print("Status:", LpStatus[model.status])
for v in model.variables():
    print(v.name, "=", v.varValue)

print("Total cost = ", value(model.objective))

Red_Tomato_Problem:
MINIMIZE
2*inventory_1 + 2*inventory_2 + 2*inventory_3 + 2*inventory_4 + 2*inventory_5 + 2*inventory_6 + 300*no_hired_1 + 300*no_hired_2 + 300*no_hired_3 + 300*no_hired_4 + 300*no_hired_5 + 300*no_hired_6 + 500*no_laidoff_1 + 500*no_laidoff_2 + 500*no_laidoff_3 + 500*no_laidoff_4 + 500*no_laidoff_5 + 500*no_laidoff_6 + 6*overtime_1 + 6*overtime_2 + 6*overtime_3 + 6*overtime_4 + 6*overtime_5 + 6*overtime_6 + 10*product_1 + 10*product_2 + 10*product_3 + 10*product_4 + 10*product_5 + 10*product_6 + 5*stockout_1 + 5*stockout_2 + 5*stockout_3 + 5*stockout_4 + 5*stockout_5 + 5*stockout_6 + 30*subcontract_1 + 30*subcontract_2 + 30*subcontract_3 + 30*subcontract_4 + 30*subcontract_5 + 30*subcontract_6 + 640*workforce_1 + 640*workforce_2 + 640*workforce_3 + 640*workforce_4 + 640*workforce_5 + 640*workforce_6 + 0
SUBJECT TO
_C1: no_hired_1 - no_laidoff_1 - workforce_1 = -80

_C2: no_hired_2 - no_laidoff_2 + workforce_1 - workforce_2 = 0

_C3: no_hired_3 - no_laidoff_3 + workf

## Problem 2
_This problem is taken from book Applied Mathematical Programming_

An iron foundry has a firm order to produce 1000 pounds of castings containing at least 0.45 percent manganese and between 3.25 percent and 5.50 percent silicon. As these particular casting are a special order, there are no suitable castings on hand. The castings sell for $0.45 per pound. The foundry has three types of pig iron available in essentially unlimited amounts, with the following propeties

| | A | B | C |
| -- | -- | -- | -- |
| Silicon | 4% | 1% | 0.6% |
| Manganese | 0.45% | 0.5% | 0.4% |

Further, the production process is such that pure manganese can also be added directly to the melt. The costs of the various possible inputs are:

* Pig A $\$$21/thousand pounds 
* Pig B $\$$25/thousand pounds
* Pig C $\$$15/thousand pounds
* Manganese $\$$8/pound

It costs 0.5 cents to melt down a pound of pig iron. Out of what inputs should the foundry produce the castings in order to maximize profit?

### Mathematical formulation

The following variables denote the corresponding number of pounds of pig A, B, C, and pure mangenese, respectively.

\begin{gather*}
x1 = \text{Thousands of pounds of pig iron A} \\
x2 = \text{Thousands of pounds of pig iron B} \\
x3 = \text{Thousands of pounds of pig iron C} \\
x4 = \text{Pounds of pure manganese}
\end{gather*}

As the castings profit per pound is $\$$0.45 and the producing quantity is fix to 1000 pounds, the total income will be as follows:

Total income = 0.45 x 1000 = 450

The total cost incurred in the production of the alloy, we should add the melting cost 0.5 cent or $\$$0.005/pound to the corresponding cost of each type of pig iron used. Thus, melting cost per thousand pounds will be 0.005 multiply to 1000 equal to $\$$5 per thousand pounds. Accordingly the unit cost of each pig iron are:

* Pig iron A   21 + 5 = 26
* Pig iron B   25 + 5 = 30
* Pig iron C   15 + 5 = 20

Therefore, the total cost becomes:

\begin{equation}
\text{Total cost} = 26x1 + 30x2 + 20x3 + 8x4,
\end{equation}

and the total profit will be:

\begin{gather*}
\text{Total profit} = \text{Total income} - \text{Total cost} \\
\text{Total profit} = 450 - 26x1 - 30x2 - 20x3 - 8x4 \\
\end{gather*}



#### Objective Function
Because the total production was fixed, maximization of the total profit is completely equivalent to the minimization of the total cost.

$$ \text{Minimize} \quad Z = 26x1 + 30x2 + 20x3 + 8x4 $$

#### Constraints
The following constraint ensure that the total amount of produced castings is exactly equal to 1000 pounds.
$$ 1000x1 + 1000x2 + 1000x3 + x4 = 1000 $$

The next constraint will ensure that the castings contain at least 0.45 percent of manganese which equal to 4.5 pounds in the 1000 pounds of produced castings. The constraint can be expressed as follows:
$$ 4.5x1 + 5x2 + 4x3 + x4 >= 4.5 $$

Similarly, the restriction regarding the silicon content, which should be in a range of 3.25 to 5.5 percent, can be represented by the following inequalities:
$$ 40x1 + 10x2 + 6x3 >= 32.5 $$
$$ 40x1 + 10x2 + 6x3 <= 55 $$

#### Solution

In [11]:
# 1st alternative
# create a model
from pulp import *
model = LpProblem("Problem 2", LpMinimize)

# create variables
x1 = LpVariable("Pig A", lowBound =0)
x2 = LpVariable("Pig B", lowBound =0)
x3 = LpVariable("Pig C", lowBound =0)
x4 = LpVariable("Manganese", lowBound =0)

# Objective function
model += 26*x1 + 30*x2 + 20*x3 + 8*x4

# constraints
#model += h1 + h2 >= 1000
model += 1000*x1 + 1000*x2 + 1000*x3 + x4 == 1000
model += 4.5*x1 + 5*x2 + 4*x3 + x4 >= 4.5
model += 40*x1 + 10*x2 + 6*x3 >= 32.5
model += 40*x1 + 10*x2 + 6*x3 <= 55

# Run the solver
model.writeLP("models/LP1-prob2-model1.lp")
model.solve()

print("Status:", LpStatus[model.status])
for v in model.variables():
    print(v.name, "=", v.varValue)

print("Total cost = ", value(model.objective))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /azhome/wapra1274@ad.uit.no/.local/lib/python3.9/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/a1f1c9b618454252bab323880d7ee336-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/a1f1c9b618454252bab323880d7ee336-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 9 COLUMNS
At line 28 RHS
At line 33 BOUNDS
At line 34 ENDATA
Problem MODEL has 4 rows, 4 columns and 14 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 4 (0) rows, 4 (0) columns and 14 (0) elements
0  Obj 0 Primal inf 33.551268 (3)
3  Obj 25.560191
Optimal - objective value 25.560191
Optimal objective 25.56019134 - 3 iterations time 0.002
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00

Status: Optimal
Manganese = 0.11072726
Pig_A = 0.7794313
Pig_B = 0.0
Pig_C